## Moku Example: Measuring microvolt signals with Moku:Pro

This example uses a custom step waveform representing Morse characters (morse_AWG.csv can be downloaded from the same Git folder as this notebook), to phase modulate a sine wave on a Moku:Pro output. With external 20 dB attenuators, the microvolt output signal is connected back to Input 1 on the same Moku:Pro device, where it is demodulated with a Lock-In Amplifier to extract phase information. The signal is also passed through a Digital Filter Box to futher filter out any unwanted noise. 

NOTE: This script was intended to run several times with variable "attenuation" values (based on the number of external attenuators and input attenuation setting) and Waveform Generator amplitude ("amp") values. A single folder called "Data" by default, is generated to store all data files where it can be processed with the MATLAB script in the same Git folder as this notebook. The duration of the logging session is specifically set to measure 10 cycles of each Morse message for post-process averaging.

For an in-depth explanation of how to measure microvolt signals on Moku:Pro, please visit the application notes section on our website.

**(c) 2025 Liquid Instruments Pty. Ltd.**

In [ ]:
import os
import time
import traceback

import numpy as np

from moku.instruments import (
    ArbitraryWaveformGenerator,
    DigitalFilterBox,
    LockInAmp,
    MultiInstrument,
    WaveformGenerator,
)

# Form connection with your Moku device; Type in your Moku's serial number
m = MultiInstrument('192.168.XX.XX', force_connect=True, platform_id=4)

# Set instrument slots in Multi-Instrument Mode
awg = m.set_instrument(slot=1, instrument=ArbitraryWaveformGenerator)
wg = m.set_instrument(slot=2, instrument=WaveformGenerator)
lia = m.set_instrument(slot=3, instrument=LockInAmp)
dfb = m.set_instrument(slot=4, instrument=DigitalFilterBox)

# Set Multi-Instrument Mode connections
connections = [
    dict(source='Slot1OutA', destination='Slot2InA'),
    dict(source='Slot2OutA', destination='Output1'),
    dict(source='Input1', destination='Slot3InA'),
    dict(source='Slot3OutA', destination='Slot4InA'),
]
m.set_connections(connections=connections)

m.set_frontend(1, impedance="50Ohm", attenuation="0dB", coupling="AC")

try:
    # Load CSV file
    lut_data = np.loadtxt('morse_AWG.csv', delimiter=',')

    # Convert to list
    lut_data_list = lut_data.tolist()

    # Use in AWG function
    awg.generate_waveform(
        channel=1, sample_rate='Auto', lut_data=lut_data_list, frequency=30e-3, amplitude=2
    )

    # Initialize the WG, set the output to 'Off' for now
    wg.generate_waveform(channel=1, type='Off')

    # Configure Lock In Amplifier settings
    lia.set_outputs(main="Theta", main_offset=0, aux='Demod', aux_offset=0)
    lia.set_polar_mode("25uVpp")
    lia.set_demodulation(mode="Internal", frequency=10e6, phase=0)
    lia.set_filter(corner_frequency=0.7, slope="Slope12dB")

    # Configure Digital Filter Box Settings
    dfb.set_control_matrix(1, input_gain1=1, input_gain2=0)
    dfb.set_filter(
        1,
        sample_rate='305.2kHz',
        shape='Lowpass',
        type='Bessel',
        low_corner=0.2,
        order=8,
        strict=False,
    )
    dfb.set_monitor(1, 'Output1')

    # Configure WG Settings
    amp = 500  # Set the amplitude of the output signal (units of mVpp)
    wg.generate_waveform(channel=1, type='Sine', frequency=10e6, amplitude=amp / 1000)
    wg.set_modulation(channel=1, type='Phase', source='InputA', depth=90)

    # Sync instrument slots
    m.sync()

    # Attenuation setting (units of dB) based on external attenuators for filename
    attenuation = 100

    # Begin logging output data
    response = dfb.start_logging(duration=400, rate=143)

    # Build a folder in the base directory and make sure it exists
    base_dir = "C:/Users/Your/Folder/Path/"  # Select the base directory where the data folder should be placed
    folder = 'Data/'  # Create the data folder name
    FILE_PATH = os.path.join(base_dir, folder)  # Generate the full file path
    os.makedirs(FILE_PATH, exist_ok=True)  # Creates the folder if missing

    # Print logging progress
    file_name = response["file_name"]
    is_logging = True
    while is_logging:
        time.sleep(10)
        progress = dfb.logging_progress()
        remaining_time = int(progress['time_remaining'])
        print(f'Time remaining: {remaining_time} s')
        is_logging = remaining_time > 1

    # When logging is finished, download the data file to the specified filepath
    temp_filename = os.path.join(
        FILE_PATH, f"{attenuation}dB_{amp}mVpp"
    )  # Name the individual data file to include the signal attenuation and amplitude
    dfb.download("ssd", file_name, temp_filename + ".li")  # Download the file to your device
    os.system(
        "mokucli convert --format=mat " + temp_filename + ".li"
    )  # Convert the .li file to a matfile
    print(
        "Downloaded log file to local directory."
    )  # Print a completion statement after each data file is downloaded

except Exception as e:
    print(f'Exception ocurred: {e}')
    traceback.print_exc()  # Prints the full traceback with line number
finally:
    print('reached relinquish ownership')
    m.relinquish_ownership()

MokuNotFound: Could not connect to Moku